In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
)
import seaborn as sns
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random
from pprint import pprint
import generate_classification_report as gcr
#def call_model(X_all, y_all, folder, device):
path1 = 'external-icelandic-reviews/data/imdb-split/hannes-reviews-reviews-with-sentiment.csv'
path2 = 'external-icelandic-reviews/data/imdb-split/kvikmyndaryni-reviews-with-sentiment.csv'

d1 = pd.read_csv(path1)
d2 = pd.read_csv(path2)
d1.drop(['num', 'rating', 'id'], axis=1, inplace=True)
d2.drop(['movie', 'rating'], axis=1, inplace=True)

print(d1.shape)
print(d2.shape)

df_orig = pd.merge(d1, d2, how='outer')

#print(df.shape)
#print(df.head())
#print(df.value_counts())

#all_pos1 = d1.where(lambda x: x['sentiment'] =='Positive')
device = 'cuda'
model = './electra-base-google-batch8-remove-noise-model/'


total = 0
for i in range(0, 10):
    r = random.randint(0, 10000)
    
    fifty_negative = df_orig.where(lambda x: x['sentiment'] == 'Negative').dropna().sample(n=50, random_state=r)
    fifty_positive = df_orig.where(lambda x: x['sentiment'] == 'Positive').dropna().sample(n=50, random_state=r)

    new_df = pd.merge(fifty_negative, fifty_positive, on=['sentiment', 'review'], how='outer')
    new_df.sentiment = new_df.sentiment.apply(lambda x: 1 if x == 'Positive' else 0)
    X_all = new_df.review
    y_all = new_df.sentiment
    accuracy = gcr.call_model(X_all, y_all, model, device)
    total += accuracy
    print('acc: {0:.4f}, seed: {1}, i: {2}'.format(accuracy, r, i))

    
print('Average accuracy: ', total/10)


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.5) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


(1111, 2)
(209, 2)


2023-11-28 02:56:33.787796: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 02:56:34.038470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 02:56:35.001355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/olafurj/.local/lib/python3.10/site-packages/trans

ValueError: cannot switch from automatic field numbering to manual field specification